<a href="https://colab.research.google.com/github/rahiakela/deep_learning_for_vision_systems/blob/5-advanced-cnn-architectures/4_inception_and_google_net.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Inception and GoogLeNet

The Inception network came to the world in 2014 when a group of researchers at Google published their paper [Going Deeper with Convolutions](https://storage.googleapis.com/pub-tools-public-publication-data/pdf/43022.pdf). The main hallmark of this architecture is building a deeper neural network while improving the utilization of the computing resources inside the network. One particular incarnation of the Inception network is called GoogleNet and was used in their submission for ILSVRC14. It uses a 22 layers deep network which is deeper than VGGNet while reducing the number of parameters 12 times
fewer (from ~138 million to ~13 million) while achieving significantly more accurate results. The network used a CNN inspired by the classical networks (AlexNet and VGGNet) but implemented a novel element which is dubbed as the Inception Module.

## Setup

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.datasets import cifar10

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Layer, Conv2D, MaxPool2D, Flatten, Dense, Input, concatenate,  Dropout, GlobalAveragePooling2D, AveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import SGD 
from tensorflow.keras.callbacks import LearningRateScheduler
from keras import backend as K

from tensorflow.keras.datasets import cifar10

import numpy as np
import pandas as pd
import math
import cv2

import matplotlib.pyplot as plt
import matplotlib.cm as cm
%matplotlib inline

Using TensorFlow backend.


## Loading and prepare CIFAR dataset

In [0]:
num_classes = 10

def load_cifar10_data(img_rows, img_cols):

    # Load cifar10 training and validation sets
    (X_train, Y_train), (X_valid, Y_valid) = cifar10.load_data()

    # Resize training images
    X_train = np.array([cv2.resize(img, (img_rows,img_cols)) for img in X_train[:,:,:,:]])
    X_valid = np.array([cv2.resize(img, (img_rows,img_cols)) for img in X_valid[:,:,:,:]])

    # Transform targets to keras compatible format
    Y_train = to_categorical(Y_train, num_classes)
    Y_valid = to_categorical(Y_valid, num_classes)
    
    X_train = X_train.astype('float32')
    X_valid = X_valid.astype('float32')

    # preprocess data
    X_train = X_train / 255.0
    X_valid = X_valid / 255.0

    return X_train, Y_train, X_valid, Y_valid

In [0]:
X_train, y_train, X_test, y_test = load_cifar10_data(224, 224)

## Novel features of Inception

The authors of the Inception network took a different approach when designing their network architecture. So, there are some architectural decisions
that we need to make for each layer when we are designing your network. Decisions like:

* **What should the kernel size of the convolutional layer be?** When designing the CONV layer, we find ourselves trying to pick and tune the kernel size of each layer that fits our dataset. As we know, smaller kernels capture finer details of the image whereas bigger filters will leave out minute details.

* **When to use the pooling layer?** AlexNet uses the pooling layers every 1 or 2 convolutional layer to downsize the spatial features. VGGNet applies pooling after every 2, 3, or 4 CONV layers as network gets deeper.

Configuring the kernel size and positioning the pool layers are decisions that we need to make mostly by trial and error and experiment with to get the optimal results. 

Inception says:**instead of choosing a desired filter size in a CONV layer and where to place the pooling layers, let’s apply all of them all together in one block and call it the “Inception Module”.**

Instead of stacking layers on top of each other like in classical architectures, the authors suggest that we create an “inception module” that consists of several convolutional layers with different kernel size. The architecture is then developed by stacking the inception modules on top of each other.

Let’s take a look at how classical convolutional networks are architected vs. the Inception network:

<img src='https://github.com/rahiakela/img-repo/blob/master/deep_learning_for_vision_systems/cnn-and-inception-network.png?raw=1' width='800'/>

From the above diagram, we can observe the following:
* In classical architectures like LeNet, AlexNet, and VGGNet, we stack convolutional and pooling layers on top of each other to build the feature extractors. At the end we add the dense FC layers to build the classifier.
* In the Inception architecture, we start with a conv + pool layers then we stack the inception modules + pooling layers to build the feature extractors then add the regular dense classifier layers.

We’ve been treating the inception modules as black boxes to understand the bigger picture of the inception architecture. Now, we will unpack the inception module to understand how it works.

## Inception module - naive version

The Inception module is a combination of four layers:

1. 1×1 Convolutional layer
2. 3×3 Convolutional layer
3. 5×5 Convolutional layer
4. 3x3 max-pooling layer

The outputs of these layers are then concatenated into a single output volume forming the input of the next stage. The naive representation of the inception module is represented in the figure below:

<img src='https://github.com/rahiakela/img-repo/blob/master/deep_learning_for_vision_systems/inception-module-1.png?raw=1' width='800'/>

The diagram might look a little overwhelming but the idea is simple to understand. Let’s follow along with this example:

1. Suppose we have an input dimensional volume from the previous layer of size =
32x32x200
2. We then feed this input to 4 convolutions simultaneously:
    * 1x1 conv with depth = 64 and same padding. The output of this kernel = 32x32x64
    * 3x3 conv with depth = 128 and same padding. Output = 32x32x128
    * 5x5 conv with depth = 32 and same padding. Output = 32x32x32
    * 3x3 max-pooling layer with same padding and stride = 1. Output = 32x32x32
3. Then, we concatenate the depth of the 4 outputs to create one output volume of dimensions = 32x32x256

Now we have an inception module that takes an input volume of 32x32x200 and outputs a volume of 32x32x256.



## Inception module with dimensionality reduction

The naive representation of the inception module that we just saw has a big computational cost problem that comes with processing larger filters like the 5x5 convolutional layer. To get a better sense of the compute problem with the naive representation, let’s calculate the number of operations that will be performed for the 5x5 CONV layer in the previous example.

The input volume with dimensions of 32x32x200 will be fed to the 5x5 conv of 32 filters with dimensions = 5x5x32. This means that total number of multiplies that the computer needs to compute is = 32x32x200 multiplied by 5x5x32 which is more than 163 million operations. While we can do this much of operations in modern computers, but this is still a pretty expensive one. This is when the dimensionality reduction layers can be very useful.

### Dimensionality reduction layer (1x1 convolutional layers):

The 1x1 convolutional layer can reduce the operational cost of 163 million operations to about a tenth of that. That is why it is given the name “reduce layer”. The idea here is to add a 1x1 CONV layer before the bigger kernels like the 3x3 and 5x5 CONV to reduce their depth which in turn will reduce the number of operations.

Let’s look at the example below:

Suppose we have an input dimension volume of 32 x 32 x 200. We then add a 1x1 CONV with depth = 16. This will reduce the dimension volume from 200 to 16 channels. We can then apply the 5x5 CONV on the output that has much less depth.

<img src='https://github.com/rahiakela/img-repo/blob/master/deep_learning_for_vision_systems/inception-module-2.png?raw=1' width='800'/>

Notice that the input of 32x32x200 is processed through the two conv layers and outputs a volume of dimensions 32x32x32 which is the same dimension that we produced before without applying the dimensionality reduction layer. But what we've done here is, instead of processing the 5x5 conv layer on the entire 200 channels of the input volume, we're taking this huge volume and shrunk its representation to a much smaller intermediate volume which only has 16 channels.

Now, let’s look at the computational cost involved in this operation and compare it to the 163 million multiplications that we got before applying the reduce layer.

```python
Computation = operations in the 1x1 convolution + operations in the 5x5 convolution
            = 32x32x200 multiplied by 1x1x16 + 32x32x16 multiplied by 5x5x32
            = 3.2 million + 13.1 million
```

Total number of multiplications in this operation = 16.3 millions which is tenth of the 163 million multiplications that we calculated earlier without the reduce layers.

### What is the impact of dimensionality reduction on the network performance?

Now we might be wondering, does shrinking down the representation size so dramatically hurt the performance of the neural network? The authors ran experimentations and found out that as long as we implement this reduce layer with moderation, we can shrink down the representation size significantly without hurting the performance and saves a lot of computation.

Now let’s put the reduce layers in action and build the new inception module with dimensionality reduction. To do that, we will keep the same concept of concatenating all the 4 layers that we had from the naive representation. We will add a 1x1 convolutional reduce layer before the 3x3 and 5x5 convolutions to reduce their computational cost. We will also add a 1x1 conv after the 3x3 max-pooling layer because pooling layers don’t reduce the depth for their inputs. So, we will need to apply the reduce layer to their output before we do the concatenation.

<img src='//github.com/moelgendy/deep_learning_for_vision_systems/raw/01f478ef7b1d9bc58c8fa8317b092e134a39ff98/chapter_05/images/inception_module.png?raw=1' width='800'/>

We add dimensionality reduction prior to bigger convolutional layers to allow for increasing the number of units at each stage significantly without an uncontrolled blow-up in computational complexity at later stages. Furthermore, the design follows the practical intuition that visual information should be processed at various scales and then aggregated so that the next stage
can abstract features from the different scales simultaneously.

## Inception architecture

Okay, now that we understand the components of the inception module, we are ready to build the inception network architecture. In here, we are going to use the dimension reduction representation of the inception module and simply stack them on top of each other and add a 3x3 pooling layer in between for downsampling as we can see in the figure below.

<img src='//github.com/moelgendy/deep_learning_for_vision_systems/raw/01f478ef7b1d9bc58c8fa8317b092e134a39ff98/chapter_05/images/inception_architecture.png?raw=1' width='800'/>

In the figure above, we stacked two inception modules with a pooling layer in between. We can stack as many inception modules as we want to build a very deep convolutional network. In the original paper, the authors built a specific incarnation of the inception module and called it **GoogLeNet**. They used this network in their submission for the ILSVRC 2014 competition.

The GoogLeNet architecture is depicted in the diagram below:

<img src='https://github.com/rahiakela/img-repo/blob/master/deep_learning_for_vision_systems/inception-module-3.png?raw=1' width='800'/>

As we can see in the diagram above, GoogLeNet uses a stack of a total of 9 inception blocks and max pooling layer after every several blocks to reduce the dimensionality. To simplify this implementation, we are going to breakdown the GoogLeNet architecture into three parts A, B, and C:

* **Part A**: identical to AlexNet and LeNet architectures where it contains a series of CONV and POOL layers
* **Part B**: contains nine inception modules stacked as follows - 2 inception modules + pooling layer + 5 inception modules + pooling layer + 2 inception modules
* **Part C**: is the classifier part of the network where it consists of the fully connected and softmax layers



## GoogleNet in Keras

Now, let’s implement GoogleNet architecture in Keras. First we build the inception module function to use it in our architecture:

<img src='//github.com/moelgendy/deep_learning_for_vision_systems/raw/01f478ef7b1d9bc58c8fa8317b092e134a39ff98/chapter_05/images/inception_module.png?raw=1' width='800'/>

Notice that the inception module takes the features from the previous module as an input, passes it through 4 routes, then concatenate the depth of the output of all 4 routes, then pass the concatenated output to the next module. The four routes are as follows:

1. 1x1 conv
2. 1x1 conv + 3x3 conv
3. 1x1 conv + 5x5 conv
4. 3x3 pool + 1x1 conv

Now, let’s build the inception_module function. The function takes the number of filters of each convolutional layer as an argument and returns the concatenated output.